In [1]:
!pip install redis
!pip install pymysql

In [2]:
import redis, json, time, pymysql
DB_Redis = redis.Redis(host='localhost', port=6379, db=0)
DB_MYSQL = pymysql.connect("localhost","tdr","tdr", "tdr")

!mysql -utdr -ptdr tdr < db.sql

def empty_mysql(table="films"):
    cur = DB_MYSQL.cursor()
    cur.execute('DELETE FROM {} WHERE 1=1'.format(table))
    DB_MYSQL.commit()
    cur.close()

def empty_redis(table = ""):
    if table.strip() == "":
        return DB_Redis.flushall()
    else:
        return DB_Redis.delete(*DB_Redis.keys("{}:*".format(table)))

cur = DB_MYSQL.cursor()
cur.execute("INSERT INTO films (title,year,genre_list,cast_list) VALUES('foo', 2019, 'bar', 'baz')")
DB_MYSQL.commit()
cur.close()
cur = DB_MYSQL.cursor()
cur.execute('SELECT title,year,genre_list,cast_list FROM films')
print(cur.fetchall())
cur.close()

mysql: [Warning] Using a password on the command line interface can be insecure.
(('foo', 2019, 'bar', 'baz'),)
True
b'bar'
1
None


In [3]:
with open('movies.json') as f:
    datas_temp = json.load(f)

#Suppression des doublons
datas_json = []
keys = []
for film in datas_temp:
    if film['title'] not in keys:
        keys.append(film['title'])
        datas_json.append(film)
print(datas_json[0])

{'title': 'After Dark in Central Park', 'year': 1900, 'cast': [], 'genres': []}


In [13]:
empty_redis()
t = time.time()
for film in datas_json:
    DB_Redis.set('films:{}'.format(film['title']), json.dumps(film))
t = time.time() - t
print('Insertions de {} films dans Redis : {:0.03f} secondes'.format(len(datas_json), t))

Insertions de 26791 films dans Redis : 2.091 secondes


In [5]:
def mysql_insert_update(sql,values=None):
    try:
        # create a new cursor
        cur = DB_MYSQL.cursor()
        if values is None:
            cur.execute(sql)
        elif isinstance(values, list):      
            # execute the INSERT statement
            cur.executemany(sql,values)
        else:
            cur.execute(sql,value)
        # commit the changes to the database
        DB_MYSQL.commit()
        # close communication with the database
        cur.close()
    except (Exception) as error:
        print(error)

In [14]:
empty_mysql()
datas_tuple = []
for film in datas_json:
    datas_tuple.append((film['title'],str(film['year']),json.dumps(film['genres']),json.dumps(film['cast'])))
print(datas_tuple[0])
t = time.time()
mysql_insert_update("INSERT INTO films(title,year,genre_list,cast_list) VALUES(%s,%s,%s,%s)", datas_tuple)
t = time.time() - t
print('Insertions de {} films dans MySQL : {:0.03f} secondes'.format(len(datas_json), t))

('After Dark in Central Park', '1900', '[]', '[]')
Insertions de 26791 films dans MySQL : 0.286 secondes


In [28]:
t = time.time()
cur = DB_MYSQL.cursor()
cur.execute("SELECT title FROM films WHERE title LIKE 'X%'")
films = list(map(lambda title : title[0], cur.fetchall()))
t = time.time() - t
print('Recherche des films commençant par A ou a avec MySQL : {} films en {:0.03f} secondes'.format(cur.rowcount, t))
print(films)
cur.close()

Recherche des films commençant par A ou a avec MySQL : 17 films en 0.005 secondes
['X Marks the Spot', 'X the Unknown', 'X-15', 'X: The Man with the X-ray Eyes', 'Xanadu', 'X-Men', 'xXx', 'X2: X-Men United', 'xXx: State of the Union', 'X-Men: The Last Stand', 'X Games 3D: The Movie', 'X-Men Origins: Wolverine', 'X-Men: First Class', 'X-Men: Days of Future Past', 'X-Men: Apocalypse', 'xXx: Return of Xander Cage', 'XX']


In [29]:
t = time.time()
films = DB_Redis.mget(DB_Redis.keys("films:[xX]*"))
films = map(lambda x : json.loads(x), films)
films = list(map(lambda x : x['title'], films))
t = time.time() - t
print('Recherche des films commençant par A ou a avec Redis : {} films en {:0.03f} secondes'.format(len(films), t))
print(films)

Recherche des films commençant par A ou a avec Redis : 17 films en 0.003 secondes
['X-15', 'X-Men: The Last Stand', 'xXx', 'X-Men', 'X-Men: First Class', 'xXx: State of the Union', 'X2: X-Men United', 'XX', 'X: The Man with the X-ray Eyes', 'X-Men: Days of Future Past', 'X the Unknown', 'Xanadu', 'X-Men: Apocalypse', 'xXx: Return of Xander Cage', 'X Games 3D: The Movie', 'X Marks the Spot', 'X-Men Origins: Wolverine']


In [7]:
t = time.time()
cur = DB_MYSQL.cursor()
cur.execute("SELECT title FROM films WHERE cast_list LIKE '%Sean Bean%'")
films = list(map(lambda title : title[0], cur.fetchall()))
t = time.time() - t
print('Recherche des films avec Sean bean avec MySQL : {} films en {:0.03f} secondes'.format(cur.rowcount, t))
print(films)
cur.close()

Recherche des films avec Sean bean avec MySQL : 15 films en 0.016 secondes
['Stormy Monday', 'Patriot Games', 'Ronin', "Don't Say a Word", 'The Lord of the Rings: The Fellowship of the Ring', 'Equilibrium', 'National Treasure', 'Flightplan', 'North Country', 'Silent Hill', 'Percy Jackson & the Olympians: The Lightning Thief', 'Mirror Mirror', 'Silent Hill: Revelation 3D', 'Jupiter Ascending', 'The Young Messiah']


In [8]:
t = time.time()
films = DB_Redis.mget(DB_Redis.keys("films:*"))
films = map(lambda x : json.loads(x), films)
films = filter(lambda x : 'Sean Bean' in x['cast'], films)
films = list(map(lambda x : x['title'], films))
t = time.time() - t
print('Recherche des films avec Sean bean avec Redis : {} films en {:0.03f} secondes'.format(len(films), t))
print(films)

Recherche des films avec Sean bean avec Redis : 15 films en 0.229 secondes
['The Lord of the Rings: The Fellowship of the Ring', 'Silent Hill', 'Silent Hill: Revelation 3D', 'Ronin', 'Jupiter Ascending', 'National Treasure', 'Stormy Monday', 'North Country', 'Patriot Games', 'Equilibrium', 'The Young Messiah', 'Flightplan', "Don't Say a Word", 'Percy Jackson & the Olympians: The Lightning Thief', 'Mirror Mirror']


In [9]:
t = time.time()
cur = DB_MYSQL.cursor()
cur.execute("SELECT title FROM films WHERE year = 2001")
films = list(map(lambda title : title[0], cur.fetchall()))
t = time.time() - t
print('Recherche des films sortis en 2001 avec MySQL : {} films en {:0.03f} secondes'.format(cur.rowcount, t))
print(films[0:10])
cur.close()

Recherche des films sortis en 2001 avec MySQL : 212 films en 0.005 secondes
['15 Minutes', '3 A.M.', '3000 Miles to Graceland', 'A.I. Artificial Intelligence', 'The Affair of the Necklace', 'Ali', 'All Over the Guy', 'Along Came a Spider', 'The Amati Girls', "America's Sweethearts"]


In [10]:
t = time.time()
films = DB_Redis.mget(DB_Redis.keys("films:*"))
films = map(lambda x : json.loads(x), films)
films = filter(lambda x : 2001 == x['year'], films)
films = list(map(lambda x : x['title'], films))
t = time.time() - t
print('Recherche des films sortis en 2001 avec Redis : {} films en {:0.03f} secondes'.format(len(films), t))
print(films[0:10])

Recherche des films sortis en 2001 avec Redis : 212 films en 0.229 secondes
['Thirteen Conversations About One Thing', 'The Pledge', 'Jurassic Park III', 'The Lord of the Rings: The Fellowship of the Ring', 'Out Cold', 'K-PAX', 'The Zeros', 'Harvard Man', 'My Big Fat Greek Wedding', '15 Minutes']


In [11]:
t = time.time()
films = DB_Redis.mget(DB_Redis.keys("films:*"))
films = list(map(lambda x : json.loads(x), films))
for film in films:
    film['year'] = film['year'] + 1
    DB_Redis.set("films:{}".format(film['title']), json.dumps(film))
t = time.time() - t
print('Mise à jour des années de sorties de {} films avec Redis en {:0.03f} secondes'.format(len(films), t))
print(json.loads(DB_Redis.get('films:Avatar')))

Mise à jour des années de sorties de 26791 films avec Redis en 2.438 secondes
{'title': 'Avatar', 'year': 2010, 'cast': ['Sam Worthington', 'Zoe Saldana', 'Sigourney Weaver', 'Michelle Rodriguez', 'Stephen Lang', 'Laz Alonso', 'Giovanni Ribisi', 'Joel Moore', 'Wes Studi'], 'genres': ['Science Fiction']}


In [12]:
t = time.time()
cur = DB_MYSQL.cursor()
cur.execute("UPDATE films SET year = year + 1 WHERE 1=1")
cur.execute("SELECT * from films")
t = time.time() - t
print('Mise à jour des années de sorties de {} avec MySQL en {:0.03f} secondes'.format(cur.rowcount, t))
cur.execute("SELECT * FROM films WHERE title = 'Avatar'")
films = list(map(lambda film : {'title': film[1], 'year': film[2], 'genre_list': json.loads(film[3]), 'cast_list': json.loads(film[4])}, cur.fetchall()))
print(films[0])
cur.close()

Mise à jour des années de sorties de 26791 avec MySQL en 0.297 secondes
{'title': 'Avatar', 'year': 2010, 'genre_list': ['Science Fiction'], 'cast_list': ['Sam Worthington', 'Zoe Saldana', 'Sigourney Weaver', 'Michelle Rodriguez', 'Stephen Lang', 'Laz Alonso', 'Giovanni Ribisi', 'Joel Moore', 'Wes Studi']}
